# Modelo Analisis de Sentimientos - NPL

Instalar librerias

In [ ]:
"""
!pip install --upgrade gspread
!pip install --upgrade gspread-dataframe
!pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2
!pip install pysentimiento
#eliminar acentos 
!pip install unidecode
#optimizar modelos de transformes en el GPU
!pip install accelerate -U
!pip install spacy
!pip install nltk
#abrir y cerrar archivos en excel
!pip install openpyxl
#autorizar google OAuth
!pip install gspread google-auth-oauthlib
"""

In [2]:
import spacy
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import pandas as pd
from unidecode import unidecode
import gspread
from gspread_dataframe import get_as_dataframe
from pysentimiento import create_analyzer
import transformers
#from google.colab import auth #solo para Colab
from google import auth
import google_auth_oauthlib

In [3]:
# Descargar herramientas de Tokenizacion

# tokenizar texto a palabras
nltk.download('punkt')
# eliminar "el", "la", etc
nltk.download('stopwords')
#complemento para mejorar la tokenizacion
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\egarcia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\egarcia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\egarcia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
# Funcion para convertir la fecha
def convertir_fecha (x):
  try: 
    if isinstance(x, (int, float)):
      return pd.to_datetime("1899-12-30") + pd.to_timedelta(int(x), "D")
    else:
      return pd.to_datetime(x, errors="coerce")
  except: 
    return pd.NaT

### Opcion 1: Importar datos de Google Sheets

File: 


https://docs.google.com/spreadsheets/d/1BHEv4qGmx2CKIlB6FaNFbps9ijmAtUrpQnVvTnSLP1A/edit?gid=1938477574#gid=1938477574


Instructions in ChatGpt for OAuth gsheets: 

https://chatgpt.com/share/68b50261-0d64-800c-b980-311043af5206

In [5]:
#Authenticate
import gspread

# First time: opens a browser window → log into your Google account
gc = gspread.oauth()

In [6]:
# Open your Google Sheet by name
sh = gc.open("7_Comentarios_Facebook")
worksheet = sh.sheet1

# Example: print all values and type
print(worksheet.get_all_values())
print(type(worksheet))

# Convert worksheet to DataFrame
# worksheet is a gspread.worksheet.Worksheet object
data = worksheet.get_all_records()   # returns list of dicts
df = pd.DataFrame(data)

print(df.head())

print(type(df))

[['Fecha', 'Comentarios'], ['1-1-2022', 'Buen servicio y el pastel llegó a tiempo ⏰'], ['1-2-2022', 'Me pareció caro para el tamaño 😬'], ['1-3-2022', 'El diseño del pastel estaba hermoso 💖'], ['1-4-2022', 'El sabor no fue lo que esperaba 😔'], ['1-5-2022', 'El sabor no fue lo que esperaba 😔'], ['1-6-2022', 'Pastel fresco y bien decorado, recomendado ✅'], ['1-7-2022', 'El pastel de zanahoria estaba espectacular 🥕'], ['1-8-2022', 'El pastel estaba muy pesado, no lo recomiendo 🙄'], ['1-9-2022', 'Se tardaron en la entrega, pero el pastel valió la pena 🎉'], ['1-10-2022', 'Los pasteles siempre están frescos y suaves 👌'], ['1-11-2022', 'Gran variedad de sabores, volveré a comprar 🤩'], ['1-12-2022', 'El mejor pastel de chocolate que he probado 🍫'], ['1-13-2022', 'Súper recomendados, todos en la fiesta quedaron felices 🎊'], ['1-14-2022', 'Me encantó el pastel, estaba delicioso 😋'], ['1-15-2022', 'El pastel de tres leches es mi favorito 🥛'], ['1-16-2022', 'Se tardaron en la entrega, pero el paste

### Opcion 2: Importar datos de Excel File

In [ ]:
file_path = '7_Comentarios_Facebook.xlsx'
df = pd.read_excel(file_path)
print(df.head())

Tratamiento de Texto

In [7]:
df['Comentarios'] = df['Comentarios'].str.lower() #convertir a minusculas
df['Comentarios'] = df['Comentarios'].apply(lambda x: unidecode(str(x))) #quitar acentos
df['Comentarios'] = df['Comentarios'].replace('', float('nan')) #vacios = nan
df['Fecha'] = df['Fecha'].replace('', float('nan')) #fechas vacias = nan
df = df.dropna(subset=['Comentarios', 'Fecha']) #quitar filas sin comentarios o fecha
df['Fecha'] = df['Fecha'].apply(convertir_fecha) #aplicar funcion de conversion 
df['Fecha'] = df['Fecha'].dt.strftime('%Y-%m-%d') #aplicando formato a YYYY-MM-DD
print(df.head())

        Fecha                                Comentarios
0  2022-01-01  buen servicio y el pastel llego a tiempo 
1  2022-01-02            me parecio caro para el tamano 
2  2022-01-03       el diseno del pastel estaba hermoso 
3  2022-01-04           el sabor no fue lo que esperaba 
4  2022-01-05           el sabor no fue lo que esperaba 


In [8]:
# evitar que transformers muestre demasiados logs al cargar el codigo de la linea siguiente 'analyzer'
transformers.logging.set_verbosity(transformers.logging.ERROR)

#crea un Analizador de Sentimiento en Espaniol (sentimiento positivo o negativo)
analyzer = create_analyzer(task='sentiment', lang='es')

In [9]:
#aplicar Tokenizacion a la columna de comentarios
#dividir oraciones "Hello Word" a palabras: "Hello",  "World"
comentarios_tokenizados = df['Comentarios'].apply(nltk.sent_tokenize)

print(comentarios_tokenizados.head())

0    [buen servicio y el pastel llego a tiempo]
1              [me parecio caro para el tamano]
2         [el diseno del pastel estaba hermoso]
3             [el sabor no fue lo que esperaba]
4             [el sabor no fue lo que esperaba]
Name: Comentarios, dtype: object


In [10]:
# analisis de sentimiento a cada comentatio
resultados_sentimientos = []
for indice, lista_oraciones in comentarios_tokenizados.items():
    sentimientos_por_oracion = []
    for oracion in lista_oraciones:
        sentimiento = analyzer.predict(oracion).output
        sentimientos_por_oracion.append(sentimiento)
    resultados_sentimientos.append(sentimientos_por_oracion)

In [11]:
#puse esto solo para entender el codigo de arriba, visualizar variables solamente
print(comentarios_tokenizados.items())
print(indice)
print(lista_oraciones)
print(sentimientos_por_oracion)
print(oracion)
print(resultados_sentimientos)

199
['pedi un pastel personalizado y quedo perfecto']
['POS']
pedi un pastel personalizado y quedo perfecto
[['POS'], ['NEG'], ['POS'], ['NEG'], ['NEG'], ['POS'], ['POS'], ['NEG'], ['POS'], ['POS'], ['POS'], ['POS'], ['POS'], ['POS'], ['POS'], ['POS'], ['POS'], ['NEG'], ['POS'], ['POS'], ['POS'], ['NEG'], ['POS'], ['POS'], ['POS'], ['NEG'], ['NEG'], ['POS'], ['POS'], ['NEG'], ['NEU'], ['POS'], ['POS'], ['NEG'], ['POS'], ['POS'], ['POS'], ['POS'], ['NEG'], ['NEG'], ['POS'], ['POS'], ['NEG'], ['POS'], ['NEU'], ['NEG'], ['NEG'], ['POS'], ['NEU'], ['POS'], ['POS'], ['NEU'], ['POS'], ['NEG'], ['POS'], ['POS'], ['POS'], ['POS'], ['NEG'], ['POS'], ['NEG'], ['NEG'], ['POS'], ['POS'], ['POS'], ['POS'], ['POS'], ['POS'], ['POS'], ['POS'], ['POS'], ['POS'], ['NEG'], ['POS'], ['POS'], ['NEU'], ['NEU'], ['NEG'], ['POS'], ['POS'], ['POS'], ['POS'], ['NEG'], ['POS'], ['POS'], ['NEU'], ['POS'], ['POS'], ['NEG'], ['POS'], ['POS'], ['NEG'], ['POS'], ['POS'], ['NEU'], ['POS'], ['POS'], ['NEG'], ['POS'], 

In [12]:
# Crear columna 'Sentimiento' y agregarla a df en la posicion 2
df.insert(loc=2, column='Sentimiento', value=resultados_sentimientos)
# convertir la lista de sentimiento en un string
df['Sentimiento'] = df['Sentimiento'].apply(lambda x: ','.join(x) if isinstance(x, list) else str(x))

In [13]:
print(df.head())
renglones = len(df)
print(renglones)
columnas = df.columns.__len__()
columnas

        Fecha                                Comentarios Sentimiento
0  2022-01-01  buen servicio y el pastel llego a tiempo          POS
1  2022-01-02            me parecio caro para el tamano          NEG
2  2022-01-03       el diseno del pastel estaba hermoso          POS
3  2022-01-04           el sabor no fue lo que esperaba          NEG
4  2022-01-05           el sabor no fue lo que esperaba          NEG
200


3

### Opcion 1: Exportar a Google Sheets

In [14]:
#open the spreadsheet
sh = gc.open("7_Comentarios_Facebook")   # same as before

#Create or select a worksheet
    #If the worksheet already exists:
#worksheet2 = sh.worksheet("NewSheet")   # existing tab
    #If it doesn’t exist, create it:
#worksheet2 = sh.add_worksheet(title="NewSheet", rows=renglones, cols=columnas)

# Write processed data to a different worksheet
try:
    worksheet2 = sh.worksheet("NewSheet")
except:
    worksheet2 = sh.add_worksheet(title="NewSheet", rows=100, cols=20)


#Clear it (optional, to avoid old data hanging around)
worksheet2.clear()

#Write your DataFrame
#Gspread doesn’t write DataFrames directly, but you can convert them to a list of lists (including headers):
values = [df.columns.tolist()] + df.values.tolist()

print(values)

# Push to Google Sheets
worksheet2.update(values)


[['Fecha', 'Comentarios', 'Sentimiento'], ['2022-01-01', 'buen servicio y el pastel llego a tiempo ', 'POS'], ['2022-01-02', 'me parecio caro para el tamano ', 'NEG'], ['2022-01-03', 'el diseno del pastel estaba hermoso ', 'POS'], ['2022-01-04', 'el sabor no fue lo que esperaba ', 'NEG'], ['2022-01-05', 'el sabor no fue lo que esperaba ', 'NEG'], ['2022-01-06', 'pastel fresco y bien decorado, recomendado ', 'POS'], ['2022-01-07', 'el pastel de zanahoria estaba espectacular ', 'POS'], ['2022-01-08', 'el pastel estaba muy pesado, no lo recomiendo ', 'NEG'], ['2022-01-09', 'se tardaron en la entrega, pero el pastel valio la pena ', 'POS'], ['2022-01-10', 'los pasteles siempre estan frescos y suaves ', 'POS'], ['2022-01-11', 'gran variedad de sabores, volvere a comprar ', 'POS'], ['2022-01-12', 'el mejor pastel de chocolate que he probado ', 'POS'], ['2022-01-13', 'super recomendados, todos en la fiesta quedaron felices ', 'POS'], ['2022-01-14', 'me encanto el pastel, estaba delicioso ', '

{'spreadsheetId': '1BHEv4qGmx2CKIlB6FaNFbps9ijmAtUrpQnVvTnSLP1A',
 'updatedRange': 'NewSheet!A1:C201',
 'updatedRows': 201,
 'updatedColumns': 3,
 'updatedCells': 603}

### Opcion 2: Exportar a Excel File

In [16]:
# exportar a excel
output_file = "7_Comentarios_Facebook_con_Sentimientos.xlsx"
df.to_excel(output_file, index=False)
print(f"Archivo exportado: {output_file}")

Archivo exportado: 7_Comentarios_Facebook_con_Sentimientos.xlsx
